In [39]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

In [2]:
nlp = spacy.load('en_core_web_md')

/Users/lukas/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [84]:
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Inches

In [175]:
bachelor_thesis = Document('./../Linss-Lukas-ba.docx')
paragraphs = bachelor_thesis.paragraphs
paragraph_dict = {}
for i, p in enumerate(paragraphs):
    if len(p.text) > 100:
        paragraph_dict[(paragraphs[i - 1].text, int(paragraphs[i - 1].style.name[-1]))] = p.text

In [176]:
paragraph_dict.keys()
#print(paragraph_dict)

dict_keys([('Abstract', 1), ('Introduction', 1), ('Objectives and Scientific Approach  ', 2), ('Literature Review', 1), ('The Metaverse', 2), ('What is the Metaverse?', 2), ('Metaverse building blocks', 2), ('Metaverse Architecture', 2), ('Metaverse Technologies', 2), ('Metaverse Platforms', 2), ('Shopping Experiences in the Metaverse', 2), ('Virtual-Commerce', 2), ('vs. e-commerce', 2), ('Influencing factors on the virtual shopping experience', 2), ('V-commerce products and spaces', 2), ('Influencing factors', 2), ('Current virtual shopping experiences', 2), ('Metaverse Development', 2), ('Building Virtual Worlds', 2), ('Virtual World Platforms & User Generated Content', 2), ('Methodology', 1), ('Context & Motivation', 2), ('The Solution', 2), ('Business Model', 2), ('Solution Development', 2), ('Development Process', 2), ('Technology Stack', 2), ('Solution Requirements', 2), ('Forms of Immersive Technology: Accessible Virtual Space', 2), ('Interface: Sense of Presence', 2), ('Interfa

In [127]:
from spacy.lang.en.stop_words import STOP_WORDS
import math

stop_words = list(STOP_WORDS)

In [128]:
def summurizer(**kwargs):
    document = kwargs.get('doc')
    doc = nlp(document)

    # Create dictionary with tokens as keys and their frequency as the value
    token_frequencies = {}
    for token in doc:
        if token.text not in stop_words:
            if token.text not in token_frequencies.keys():
                token_frequencies[token.text] = 1
            else:
                token_frequencies[token.text] += 1
    
    # Divide each token by the maximum frequency to get the waited frequency of each token
    for token in token_frequencies:
        token_frequencies[token] = token_frequencies[token] / max(token_frequencies.values())

    # Create dictionary with sentence as key and scentence score as value
    scentence_list = [s for s in doc.sents]
    scentence_score = {}

    # sentence score is equal to sum of token scores for each token in the scentence
    for sentence in scentence_list:
        for token in sentence:
            if token.text.lower() in token_frequencies.keys():
                if len(sentence.text.split(' ')) < 30:
                    if sentence not in scentence_score.keys():
                        scentence_score[sentence] = token_frequencies[token.text.lower()]
                    else:
                        scentence_score[sentence] += token_frequencies[token.text.lower()]

    # Summurize documents to n-sentences
    n = math.floor(kwargs.get('ratio') * len(scentence_list))

    summurized_sentences = nlargest(n, scentence_score, key=scentence_score.get)
    summary = [s for s in scentence_score.keys() if s in summurized_sentences]
    new_text = ' '.join([s.text for s in summary])
    
    print(len(document), '-', len(new_text),'=',f'{len(document) - len(new_text)} words saved')

    return new_text

In [57]:
def extractKeywords(**kwargs):
    doc = nlp(kwargs.get('doc'))
    tags = ['PROPN', 'ADJ', 'NOUN']
    keywords = {}

    for token in doc:
        if token.text not in stop_words and token.text not in punctuation:
            if token.pos_ in tags:
                if token.text in keywords.keys():
                    keywords[token.text] += 1
                else:
                    keywords[token.text] = 1
        
    return nlargest(kwargs.get('n'), keywords, key=keywords.get)

In [ ]:
from spacytextblob.spacytextblob import SpacyTextBlob

# The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.
nlp.add_pipe('spacytextblob')

In [205]:
def sentimentAnalysis(**kwargs):
    doc = nlp(kwargs.get('doc'))
    return doc._.assessments

In [206]:
sentimentAnalysis(doc=list(paragraph_dict.values())[0])

[(['becoming'], 0.45, 0.8500000000000001, None),
 (['becoming'], 0.45, 0.8500000000000001, None),
 (['promising'], 0.2, 0.5, None),
 (['new'], 0.13636363636363635, 0.45454545454545453, None),
 (['such'], 0.0, 0.5, None),
 (['new'], 0.13636363636363635, 0.45454545454545453, None),
 (['many'], 0.5, 0.5, None),
 (['clear'], 0.10000000000000002, 0.3833333333333333, None),
 (['present'], 0.0, 0.0, None),
 (['academic'], 0.0, 0.0, None),
 (['other'], -0.125, 0.375, None),
 (['commercial'], 0.0, 0.0, None),
 (['traditional'], 0.0, 0.75, None),
 (['many'], 0.5, 0.5, None),
 (['interested'], 0.25, 0.5, None),
 (['new'], 0.13636363636363635, 0.45454545454545453, None),
 (['several'], 0.0, 0.0, None),
 (['many'], 0.5, 0.5, None),
 (['tries'], -0.1, 0.4, None),
 (['light'], 0.4, 0.7, None),
 (['following'], 0.0, 0.1, None),
 (['easy'], 0.43333333333333335, 0.8333333333333334, None),
 (['initial'], 0.0, 0.0, None),
 (['multiple'], 0.0, 0.0, None),
 (['such'], 0.0, 0.5, None),
 (['various'], 0.0, 0.

In [120]:
extractKeywords(doc=list(paragraph_dict.values())[0], n=5)

['virtual', 'metaverse', 'store', 'channel', 'shopping']

In [121]:
summurizer(doc=list(paragraph_dict.values())[0], ratio=0.5)

2012 - 793 = 1219 words saved


'While many brands are interested in utilising this new channel, creating them comes with several challenges and uncertainties as many companies lack experience within this field. The scope of this research included the initial requirements analysis, followed by the implementation, and launch of the solution. Throughout the development process, multiple requirements such as resource management, modular design, and payment integration were identified and implemented. Additionally, various store atmospheric elements were utilised in the design of the user interface. The resulting solution was successfully launched and was able to attract eight customers who leased a store within the mall. Furthermore, the mall was visited by 145 users within the first three weeks following the launch.\n'

In [184]:
with open('summurized_text.txt', 'w') as t:

    document = Document()
    wc_picture = document.add_picture('./../WordCloud/word_cloud.png', width=Inches(4), height=Inches(4))
    pic_paragraph = document.paragraphs[-1] 
    pic_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

    title = document.add_heading('Shopping Experiences in the Metaverse:', 0)
    sub_title = document.add_heading('Implementation of a white lable Metaverse Solution', 1)
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER
    sub_title.alignment = WD_ALIGN_PARAGRAPH.CENTER
    document.add_page_break()


    i = 0
    for k, v in paragraph_dict.items():
    
        document.add_heading(k[0], k[1])
        paragraph = document.add_paragraph(summurizer(doc=v, ratio=0.5))
        paragraph.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
        document.add_paragraph(f'Keywords: { extractKeywords(doc=v, n=4) }')
        document.add_paragraph()
        #document.add_page_break()
        i += 1
    
    document.save('ba_summary.docx')



2012 - 793 = 1219 words saved
2709 - 917 = 1792 words saved
1719 - 789 = 930 words saved
767 - 303 = 464 words saved
3801 - 786 = 3015 words saved
4893 - 1592 = 3301 words saved
829 - 315 = 514 words saved
2930 - 1177 = 1753 words saved
7969 - 3855 = 4114 words saved
3340 - 1565 = 1775 words saved
994 - 327 = 667 words saved
3778 - 1723 = 2055 words saved
5181 - 2568 = 2613 words saved
731 - 227 = 504 words saved
1744 - 781 = 963 words saved
8290 - 3787 = 4503 words saved
4007 - 1754 = 2253 words saved
484 - 168 = 316 words saved
5009 - 2265 = 2744 words saved
3466 - 1403 = 2063 words saved
853 - 311 = 542 words saved
2491 - 1136 = 1355 words saved
4466 - 2186 = 2280 words saved
4256 - 1820 = 2436 words saved
534 - 260 = 274 words saved
2534 - 1011 = 1523 words saved
5460 - 2556 = 2904 words saved
506 - 254 = 252 words saved
523 - 274 = 249 words saved
624 - 223 = 401 words saved
967 - 388 = 579 words saved
1261 - 483 = 778 words saved
987 - 453 = 534 words saved
714 - 286 = 428 words 